In [ ]:
import pandas as pd
import geopandas as gpd
from plotnine import *
import numpy as np
import plotly.express as px

### Analysis of Dengue cases in Mexico



In [ ]:
mx_dengue_df = pd.read_pickle("../data/mexico_dengue.pkl")
mx_municipalities_df = pd.read_csv("../tablas_de_catologo/Municipalities.csv")
mx_states_df = pd.read_csv("../tablas_de_catologo/States.csv")
mx_admin_bound_sf = gpd.read_file("../mex_admbnda_govmex_20200618_shp/mex_admbnda_adm1_govmex_20200618.dbf")
mx_states_df = mx_states_df.assign(state_admin_code = lambda x: "MX" + x["ENTITY"].astype(str).str.zfill(2))

In [ ]:
mx_admin_bound_sf.info()

In [ ]:
ggplot(mx_admin_bound_sf) +\
            geom_map(aes(geom = "geometry"), fill = "none") +\
                theme(dpi = 250, 
                      axis_text= element_blank(), 
                      axis_ticks = element_blank(),
                      )

In [ ]:
mx_municipalities_states_df = mx_municipalities_df.\
            merge(mx_states_df, how = "left", left_on = "STATE", right_on = "ENTITY").drop(columns = "ENTITY")
mx_municipalities_states_df

In [ ]:
mx_dengue_df.info()

In [ ]:
logic_dict = {1:1,
               2:0}
sex_dict = {1:"FEMALE",
           2:"MALE",
           99:"NOT_SPECIFIED"}

patient_dict = {1:"AMBULATORY",
                2:"HOSPITALIZED",
                3:"UNSPECIFIED"}

dictamen_dict = {1:"CONFIRMED_DENGUE",
                 2: "CONFIRMED_CHIKUNGUNYA",
                 3: "NEGATIVE",
                 4: "IN_ANALYSIS",
                 5: "NOT_APPLICABLE"}

pcr_result_dict = {1: "DENV1",
                   2: "DENV2",
                   3: "DENV3",
                   4: "DENV4",
                   5: "NO_ISOLATED_SEROTYPE"}

stat_case_dict = {1:"PROBABLE",
                  2: "CONFIRMED",
                  3: "DISCARDED"}

In [ ]:
mx_dengue_df.DICTAMEN.value_counts()

In [ ]:
mx_dengue_cleaned_df = mx_dengue_df.assign(SEXO = lambda x: x["SEXO"].map(sex_dict),
                    HABLA_LENGUA_INDIG = lambda x: x["HABLA_LENGUA_INDIG"].map(logic_dict),
                    INDIGENA = lambda x: x["INDIGENA"].map(logic_dict),
                    TIPO_PACIENTE = lambda x: x["TIPO_PACIENTE"].map(patient_dict),
                    HEMORRAGICOS = lambda x: x["HEMORRAGICOS"].map(logic_dict),
                    DIABETES = lambda x: x["DIABETES"].map(logic_dict),
                    HIPERTENSION = lambda x: x["HIPERTENSION"].map(logic_dict),
                   ENFERMEDAD_ULC_PEPTICA = lambda x: x["ENFERMEDAD_ULC_PEPTICA"].map(logic_dict),
                   ENFERMEDAD_RENAL = lambda x: x["ENFERMEDAD_RENAL"].map(logic_dict), 
                   INMUNOSUPR = lambda x: x["INMUNOSUPR"].map(logic_dict),
                    CIRROSIS_HEPATICA = lambda x: x["CIRROSIS_HEPATICA"].map(logic_dict),
                    EMBARAZO = lambda x: x["EMBARAZO"].map(logic_dict),
                    DEFUNCION = lambda x: x["DEFUNCION"].map(logic_dict),
                    DICTAMEN = lambda x: x["DICTAMEN"].map(dictamen_dict),
                   TOMA_MUESTRA = lambda x: x["TOMA_MUESTRA"].map(logic_dict),
                   RESULTADO_PCR = lambda x: x["RESULTADO_PCR"].map(pcr_result_dict),
                   ESTATUS_CASO = lambda x: x["ESTATUS_CASO"].map(stat_case_dict))


In [ ]:
mx_dengue_cleaned_df = mx_dengue_cleaned_df.merge(mx_municipalities_states_df, 
                               how = "left", 
                               left_on = ["ENTIDAD_UM_NOTIF", "MUNICIPIO_UM_NOTIF"], 
                               right_on = ["STATE","MUNICIPALITY_INDEX"]).\
                    assign(MUNICIPIO_UM_NOTIF = lambda x: x["MUNICIPALITY"], 
                           ENTIDAD_UM_NOTIF = lambda x: x["FEDERAL_ENTITY"],
                           ADMIN_CODIGO_NOTIF = lambda x: x["state_admin_code"]).\
                    drop(columns = ["MUNICIPALITY_INDEX",
                                    "MUNICIPALITY",
                                    "STATE",
                                    "FEDERAL_ENTITY",
                                    "ABBREVIATION",
                                    "state_admin_code"])

In [ ]:
mx_dengue_cleaned_df = mx_dengue_cleaned_df.merge(mx_municipalities_states_df, 
                               how = "left", 
                               left_on = ["ENTIDAD_ASIG", "MUNICIPIO_ASIG"], 
                               right_on = ["STATE","MUNICIPALITY_INDEX"]).\
                    assign(MUNICIPIO_ASIG = lambda x: x["MUNICIPALITY"], 
                            ENTIDAD_ASIG = lambda x: x["FEDERAL_ENTITY"],
                           ADMIN_CODIGO_ASIG = lambda x: x["state_admin_code"]).\
                    drop(columns = ["MUNICIPALITY_INDEX",
                                    "MUNICIPALITY",
                                    "STATE",
                                    "FEDERAL_ENTITY",
                                    "ABBREVIATION",
                                    "state_admin_code"])

In [ ]:
mx_dengue_cleaned_df = mx_dengue_cleaned_df.assign(FECHA_SIGN_SINTOMAS = lambda x: pd.to_datetime(x["FECHA_SIGN_SINTOMAS"])).\
                     assign(MES_SIGN_SINTOMAS = lambda x:\
                                pd.to_datetime("01-" +\
                                                x["FECHA_SIGN_SINTOMAS"].dt.month.astype(str) +\
                                                   "-" +  x["FECHA_SIGN_SINTOMAS"].dt.year.astype(str), dayfirst = True))

In [ ]:
mx_dengue_monthly_count = mx_dengue_cleaned_df.groupby(["MES_SIGN_SINTOMAS", "DEFUNCION"], as_index = False).agg(NUM_CASOS = pd.NamedAgg("MES_SIGN_SINTOMAS", "count")).assign(DEFUNCION = lambda x: x["DEFUNCION"].astype(str))

In [ ]:
ggplot(mx_dengue_monthly_count, aes(x = "MES_SIGN_SINTOMAS", y = "NUM_CASOS", fill = "DEFUNCION")) +\
                    geom_col() +\
                    scale_y_log10() +\
                    theme_538() +\
                    theme(axis_text_x= element_text(angle = 30)) +\
                    xlab("Month Symptoms Reported") +\
                    ylab("Number of Cases With Symptoms")
                    

In [ ]:
ggplot(mx_dengue_cleaned_df.loc[:,["EDAD_ANOS", "DEFUNCION"]].assign(DEFUNCION = lambda x: x["DEFUNCION"].astype(str)), aes(y = "EDAD_ANOS", x = "DEFUNCION")) + geom_boxplot()

In [ ]:
mx_dengue_cleaned_df.query("ESTATUS_CASO != 'DISCARDED'").groupby(["SEXO", "DEFUNCION"], as_index = False).agg(count = pd.NamedAgg("DEFUNCION", "count"))

In [ ]:
mx_dengue_cleaned_df

In [ ]:
#mx_dengue_state_count_df = mx_admin_bound_sf.merge(mx_dengue_state_count_df, left_on = "ADM1_PCODE", right_on = "ADMIN_CODIGO_NOTIF", how = "left")
#mx_dengue_state_count_df.head()

ggplot() +\
        geom_map(mx_dengue_state_count_df, aes(geom = "geometry",
                                               fill = "num_case",
                                               color = "num_case")) +\
        scale_fill_gradient(low = "#d1d1d1", high = "blue") +\
        scale_color_gradient(low = "#d1d1d1", high = "blue") +\
                theme(dpi = 250, 
                      axis_text= element_blank(), 
                      axis_ticks = element_blank(),
                      panel_background= element_blank())

In [ ]:
mx_dengue_cleaned_df

In [ ]:
mx_dengue_state_count_df = mx_dengue_cleaned_df.\
                groupby(["ADMIN_CODIGO_NOTIF","ENTIDAD_UM_NOTIF"], as_index= False).\
agg(num_case = pd.NamedAgg("ADMIN_CODIGO_NOTIF", "count"))

mx_dengue_boundary_count_df = mx_admin_bound_sf.\
            merge(mx_dengue_state_count_df,
                  left_on = "ADM1_PCODE",
                  right_on = "ADMIN_CODIGO_NOTIF",
                  how = "left")




In [ ]:
mx_dengue_boundary_count_df

In [ ]:
import plotly.express as px
## Daily Counts
mx_dengue_cuenta_df = mx_dengue_cleaned_df.groupby(["MES_SIGN_SINTOMAS", "DEFUNCION"], as_index = False).\
            agg(CUENTA_CASOS = pd.NamedAgg("MES_SIGN_SINTOMAS", "count")).\
                assign(DEFUNCION = lambda x: np.where(x["DEFUNCION"] == 1, "SI", "NO"))

dengue_fig = px.bar(mx_dengue_cuenta_df,\
                        x = "MES_SIGN_SINTOMAS",\
                        y = "CUENTA_CASOS",\
                        color = "DEFUNCION",\
                        facet_col = "DEFUNCION",\
                        log_y= True)

dengue_fig.show()

In [ ]:
import plotly.express as px
data_canada = px.data.gapminder().query("country == 'Canada'")
fig = px.bar(data_canada, x='year', y='pop')
fig

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json as json

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

summary_string = "This is a summary string that I'm going to use to insert a summary and any pertinent instructions into the top of the dashboard I'll put a break here:  \
does it work?"

mx_dengue_cuenta_df = mx_dengue_cleaned_df.groupby(["ENTIDAD_UM_NOTIF","MES_SIGN_SINTOMAS", "DEFUNCION"], as_index = False).\
            agg(CUENTA_CASOS = pd.NamedAgg("MES_SIGN_SINTOMAS", "count")).\
                assign(DEFUNCION = lambda x: np.where(x["DEFUNCION"] == 1, "SI", "NO"))

mx_dengue_cuenta_geo_df = mx_dengue_cleaned_df

fig_map = px.choropleth_mapbox(mx_dengue_boundary_count_df,
              geojson= mx_dengue_boundary_count_df.geometry,
              locations = mx_dengue_boundary_count_df.index,
              hover_name = "ENTIDAD_UM_NOTIF",
              color_continuous_scale = "agsunset",
              color = "num_case",
              center={"lat": 23.75, "lon": -99.1200},
              mapbox_style = "carto-positron",
              zoom = 3,
              width = 500)
fig_map.update_layout(clickmode = "event+select")

fig_map.update_geos(fitbounds="locations", visible=False)

# Build App
app = JupyterDash(__name__)
app.layout = html.Div(children = [
    dcc.Markdown(summary_string),
    
    html.H1("Dengue Cases In Mexico",style = {"textAlign":"center"}),
    
    html.Div(dcc.Graph(figure = fig_map,
                       id = "mexico-map")
            ),
    
    dcc.Graph(id='graph'),
    
    html.Label([
        
        html.H4("Select State"),
        
        dcc.Dropdown(
            id='entidad_notif', clearable=False,
            value='ENTIDAD_DEFAULT', options=[
                {'label': c, 'value': c}
                for c in mx_dengue_cuenta_df.ENTIDAD_UM_NOTIF.drop_duplicates().to_list()
            ])
    ]), 
    html.Div([
            dcc.Markdown("""
                **Click Data**

                Click on points in the graph.
            """),
            html.Pre(id='click-data', style=styles['pre']),
        ], className='three columns'),
])

# Define callback to update graph

@app.callback(
    Output('click-data', 'children'),
    Input('mexico-map', 'selectedData'))

def display_click_data(selectedData):
    if selectedData == None:
        return "All"
    else:
        return selectedData["points"][0]["hovertext"]


@app.callback(
    Output('graph', 'figure'),
    [Input("mexico-map", "selectedData")]
)


def update_figure(selectedData):
    if selectedData != None:
        mx_dengue_tmp_df = mx_dengue_cuenta_df[mx_dengue_cuenta_df["ENTIDAD_UM_NOTIF"] == selectedData["points"][0]["hovertext"]]
    else:
        mx_dengue_tmp_df = mx_dengue_cuenta_df.groupby(["MES_SIGN_SINTOMAS", "DEFUNCION"], as_index = False).\
            agg(CUENTA_CASOS = pd.NamedAgg("CUENTA_CASOS", np.sum))
    fig =  px.bar(mx_dengue_tmp_df,\
                        x = "MES_SIGN_SINTOMAS",\
                        y = "CUENTA_CASOS",\
                        color = "DEFUNCION",\
                        facet_col = "DEFUNCION",\
                        log_y= True)
    return fig
# Load Data
# Run app and display result inline in the notebook
app.run_server(mode="external", port = 8080)

In [ ]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import json as json

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}


mx_dengue_cuenta_df = mx_dengue_cleaned_df.groupby(["ENTIDAD_UM_NOTIF","MES_SIGN_SINTOMAS", "DEFUNCION"], as_index = False).\
            agg(CUENTA_CASOS = pd.NamedAgg("MES_SIGN_SINTOMAS", "count")).\
                assign(DEFUNCION = lambda x: np.where(x["DEFUNCION"] == 1, "SI", "NO"))

mx_dengue_cuenta_geo_df = mx_dengue_cleaned_df

fig_map = px.choropleth_mapbox(mx_dengue_boundary_count_df,
              geojson= mx_dengue_boundary_count_df.geometry,
              locations = mx_dengue_boundary_count_df.index,
              hover_name = "ENTIDAD_UM_NOTIF",
              color_continuous_scale = "agsunset",
              color = "num_case",
              center={"lat": 23.75, "lon": -99.1200},
              mapbox_style = "carto-positron",
              zoom = 3,
              width = 500)
fig_map.update_layout(clickmode = "event+select")

fig_map.update_geos(fitbounds="locations", visible=False)

df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div(children = [
    html.H1("Dengue Cases In Mexico",style = {"textAlign":"center"}),
    
    html.Div(dcc.Graph(figure = fig_map,
                       id = "mexico-map")
            ),
    
    dcc.Graph(id='graph'),
    
    html.Label([
        
        html.H4("Select State"),
        
        dcc.Dropdown(
            id='entidad_notif', clearable=False,
            value='ENTIDAD_DEFAULT', options=[
                {'label': c, 'value': c}
                for c in mx_dengue_cuenta_df.ENTIDAD_UM_NOTIF.drop_duplicates().to_list()
            ])
    ]), 
    html.Div([
            dcc.Markdown("""
                **Click Data**

                Click on points in the graph.
            """),
            html.Pre(id='click-data', style=styles['pre']),
        ], className='three columns'),
])

# Define callback to update graph

@app.callback(
    Output('click-data', 'children'),
    Input('mexico-map', 'clickData'))

def display_click_data(clickData):
    if clickData == None:
        return "All"
    else:
        return clickData["points"][0]["hovertext"]


@app.callback(
    Output('graph', 'figure'),
    [Input("entidad_notif", "value")]
)


def update_figure(entidad):
    if entidad != "ENTIDAD_DEFAULT":
        mx_dengue_tmp_df = mx_dengue_cuenta_df[mx_dengue_cuenta_df["ENTIDAD_UM_NOTIF"] == entidad]
    else:
        mx_dengue_tmp_df = mx_dengue_cuenta_df.groupby(["MES_SIGN_SINTOMAS", "DEFUNCION"], as_index = False).\
            agg(CUENTA_CASOS = pd.NamedAgg("CUENTA_CASOS", np.sum))
    fig =  px.bar(mx_dengue_tmp_df,\
                        x = "MES_SIGN_SINTOMAS",\
                        y = "CUENTA_CASOS",\
                        color = "DEFUNCION",\
                        facet_col = "DEFUNCION",\
                        log_y= True)
    return fig
# Load Data
# Run app and display result inline in the notebook
app.run_server(mode="external", port = 9030)